## Download github repo

In [1]:
#just clone the repo
!git clone https://github.com/nhvd3500111/ditto

In [2]:
#In case you want to reupload ditto if you have made changes in the github
'''
import shutil
shutil.rmtree("/kaggle/working/ditto")
%cd /kaggle/working/
!git clone https://github.com/nhvd3500111/ditto
%cd /kaggle/working/ditto
'''


## Install packages and fp16 optimization

In [3]:
# install requirements
%cd /kaggle/working/ditto
!pip install -r requirements.txt
import nltk
import os
import time
nltk.download('stopwords')
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..
# some issue with colab
!pip install --upgrade "urllib3==1.25.4" awscli
%cd /kaggle/working/ditto
!pip install transformers
!pip install tensorboardX
!pip install jsonlines
!pip install openpyxl

## Preprocess - define the specific details of each run

In [4]:
datasets = ["all","computers", "cameras", "shoes", "watches"]
sizes = ["small", "medium"]
neurals = ["linear","cls_sep","cls_sep_gru","gru","lstm"]
gpu_id = 0

## Train DITTO - Run Matcher


In [ ]:
for run_id in range(3):
    for neural in neurals:
        for d in datasets:
            for size in sizes:
                dataset = '_'.join(['wdc', d, size])
                #for dk in [True, False]:
                    #for da in [True, False]:
                cmd = """CUDA_VISIBLE_DEVICES=%d python train_ditto.py \
                  --task %s \
                  --batch_size 32 \
                  --max_len 128 \ 
                  --lr 3e-5 \
                  --n_epochs 20 \
                  --finetuning \
                  --save_model \
                  --run_id %d \
                  --da del \
                  --neural %s""" % (gpu_id, dataset, run_id,neural)  
            
                

                ## if dk:
                    # cmd += ' --dk product'
                #if attr != 'title':
                    #cmd += ' --summarize'
                print(cmd)
                os.system(cmd)


                cmd = """CUDA_VISIBLE_DEVICES=%d  python matcher.py \
                  --task %s \
                  --input_path data/wdc/%s/test.txt \
                  --output_path output/output_small_1.jsonl \
                  --use_gpu \
                  --n_epochs 20 \
                  --da del \
                  --checkpoint_path checkpoints/ \
                  --neural %s\ 
                  --file_excel F1_SCORES.xlsx """ % (gpu_id, dataset,d,neural)

                print(cmd)
                os.system(cmd)

In [ ]:
# Step 1:
# train the Ditto model_GRU (pre_trained lm=distilbert) with the save_model flag on. 
#It will produce two files *_dev.pt and *_test.pt which are the 
# best checkpoints on the dev set and the test set.

!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task wdc_computers_small \
  --batch_size 32 \
  --max_len 128 \
  --lr 3e-5 \
  --n_epochs 20 \
  --finetuning \
  --lm distilbert \
  --save_model \
  --da entry_swap \
  --neural lstm 


## Running the matcher

In [ ]:
# Step 2: run the matcher

#da , dk and summarize must be the same as above trained
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task wdc_computers_small \
  --input_path data/wdc/computers/test.txt \
  --output_path output/output_small_1.jsonl \
  --lm distilbert \
  --max_len 128 \
  --use_gpu \
  --da entry_swap\
  --checkpoint_path checkpoints/ \
  --neural lstm \
  --file_excel F1_SCORES.xlsx

#We will download the xlsx file manually from the output folder